# Notebook lecture 13: MPC reloaded
&copy; 2025 ETH Zurich, Joël Gmür, Joël Lauper, Niclas Scheuer, Dejan Milojevic; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

Authors:
- Joël Gmür; jgmuer@ethz.ch
- Joël Lauper; jlauper@ethz.ch
- more (please add your name)

## Description
This week's Jupyter notebook will include the continuous and discrete LQR and the MPC tracking problem including costs and constraints.

To start, run the following cell to install the necessary modules and import the libraries.

In [ ]:
!pip install numpy scipy matplotlib ipywidgets control

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets
import control as ctrl
from IPython.display import display, clear_output

np.set_printoptions(suppress=True, precision=3)

# Exercise 1: LQR
### Code the linearized matrices
Consider the continuous-time linearized model of the a unmanned Areal Vehicle (UAV) derived in the exercise set:

$$
\dot x = A\,x + B\,u,
$$

$$
A = 
        \begin{bmatrix}
            0 & 0 & 0 & -V\sin(\xi_{ref}) & 0 \\
            0 & 0 & 0 & V \cos(\xi_{ref}) & 0 \\
            0 & 0 & 0 & 0 & 1 \\
            0 & 0 & \frac{g}{V} & 0 & 0 \\
            0 & 0 & -a_0 & 0 & -a_1
        \end{bmatrix}, \quad B = 
        \begin{bmatrix}
            0 \\ 0 \\ 0 \\ 0 \\ b_0
        \end{bmatrix}.
$$

Assume that $$

In [ ]:
#Code in the linearized matrices
V = 0 #CHANGE
xi = 0 #CHANGE
g = 9.81 #gravity
a_0 = 0 #CHANGE
a_1 = 0 #CHANGE
b_0 = 0 #CHANGE


A = np.array([[0,0,0,-V*np.sin(xi),0],
              [0,0,0,V*np.cos(xi),0],
              [0,0,0,0,1],
              [0,0,g/V,0,0],
              [0,0,-a_0,0,-a_1]])

B = np.array([[0],
              [0],
              [0],
              [0],
              [b_0]])

### Code in the Q, R matrices and the CLQR solution
Use Ackermann's Formula

In [ ]:
Q = np.diag([10, 1, 10, 1, 100, 10]) #EXAMPLE
R = np.eye(2) #EXAMPLE

from scipy.linalg import solve_continuous_are

def lqr(A, B, Q, R):
    P = solve_continuous_are(A, B, Q, R)
    K = np.linalg.inv(R) @ B.T @ P
    return K

K_ct = lqr(A, B, Q, R)

### Code in the DLQR solution
Choose a dicretization method and code it as a function

In [ ]:
def discretize_euler_forward(A, B, C, T):
    A_d = np.eye(A.shape[0]) + T * A
    B_d = T * B
    C_d = C
    return A_d, B_d, C_d

def discretize_euler_backward(A, B, C, T):
    A_d = np.linalg.inv(np.eye(A.shape[0]) - T * A)
    B_d = T * A_d @ B
    C_d = C
    return A_d, B_d, C_d

def discretize_trapezoidal(A, B, C, h):
    I = np.eye(A.shape[0])
    A_d = np.linalg.inv(I - 0.5 * h * A) @ (I + 0.5 * h * A)
    B_d = np.linalg.inv(I - 0.5 * h * A) @ (h * B)
    C_d = C.copy() if C is not None else None
    return A_d, B_d, C_d

def discretize_exact(A, B, C, h):
    n = A.shape[0]
    m = B.shape[1]
    M = np.zeros((n + m, n + m))
    M[:n, :n] = A
    M[:n, n:] = B
    expM = expm(h * M)
    Ad = expM[:n, :n]
    Bd = expM[:n, n:]
    Cd = C.copy() if C is not None else None
    return Ad, Bd, Cd

Define the DLQR controller

In [ ]:
from scipy.linalg import solve_discrete_are

def dlqr(A_d, B_d, Q, R):
    P = solve_discrete_are(A_d, B_d, Q, R)
    K = np.linalg.inv(B_d.T @ P @ B_d + R) @ (B_d.T @ P @ A_d)
    return K

### Code the MPC tracking problem, cost and constraints